## VGG16 모델을 이용하여 CAT, DOG 분류해보기

In [1]:
from google.colab import files
up = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
import os, json

with open('kaggle.json') as f:
  kaggle = json.load(f)

print(kaggle)

{'username': 'haminjupark', 'key': 'a204e798d6b092508c03b70d28a19f36'}


In [3]:
os.environ['KAGGLE_USERNAME'] = kaggle['username']
os.environ['KAGGLE_KEY'] = kaggle['key']

In [4]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset

 97% 764M/788M [00:04<00:00, 226MB/s]
100% 788M/788M [00:04<00:00, 167MB/s]


In [5]:
!unzip microsoft-catsvsdogs-dataset.zip > filenames

In [6]:
cat_dir = 'PetImages/Cat'
os.unlink(f'{cat_dir}/Thumbs.db')       # 압축 해제시 생성된 임시파일 삭제
cat_list = os.listdir(cat_dir)
len(cat_list)

12500

In [7]:
dog_dir = 'PetImages/Dog'
os.unlink(f'{dog_dir}/Thumbs.db')       # 압축 해제시 생성된 임시파일 삭제
dog_list = os.listdir(dog_dir)
len(dog_list)

12500

In [8]:
import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from glob import glob

In [9]:
# Removing error file
!cp PetImages/Cat/9999.jpg PetImages/Cat/666.jpg
!cp PetImages/Dog/9999.jpg PetImages/Dog/11702.jpg

In [10]:
!mkdir PetImages/test PetImages/valid
!mkdir PetImages/test/cats PetImages/test/dogs
!mkdir PetImages/valid/cats PetImages/valid/dogs

!mv PetImages/Cat/8???.* PetImages/valid/cats
!mv PetImages/Dog/8???.* PetImages/valid/dogs
!mv PetImages/Cat/9???.* PetImages/valid/cats
!mv PetImages/Dog/9???.* PetImages/valid/dogs

!mv PetImages/Cat/1????.* PetImages/test/cats
!mv PetImages/Dog/1????.* PetImages/test/dogs

!mkdir PetImages/train
!mv PetImages/Cat PetImages/train/cats
!mv PetImages/Dog PetImages/train/dogs

- 모델 구성

In [17]:
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [25]:
base_model = VGG16()
base_model.summary()
predictions_layer = base_model.get_layer('predictions')
num_classes = predictions_layer.output_shape[1]
# new_predictions = Dense(num_classes, activation='sigmoid', name='new_predictions')(base_model.layers[-2].output)
new_predictions = Dense(1, activation='sigmoid', name='new_predictions')(base_model.layers[-2].output)
custom_vgg16 = Model(inputs=base_model.input, outputs=new_predictions, name='custom_vgg16')
custom_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [26]:
custom_vgg16.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

model_path = 'best-cats-and-dogs.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)
valid_datagen = ImageDataGenerator(rescale=1/255.)

In [28]:
train_dir = 'PetImages/train'       # 8000 + 8000
test_dir = 'PetImages/test'         # 2500 + 2500
valid_dir = 'PetImages/valid'       # 2000 + 2000

In [29]:
train_generator = train_datagen.flow_from_directory(
    train_dir, batch_size=20, class_mode='binary', target_size=(244,244)
)
test_generator = test_datagen.flow_from_directory(
    test_dir, batch_size=20, class_mode='binary', target_size=(244,244)
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir, batch_size=20, class_mode='binary', target_size=(244,244)
)

Found 16000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [31]:
hist = custom_vgg16.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=100,
    steps_per_epoch=800,    # batch_size(20) x steps_per_epoch(800) = 16000, 트레인 데이터가 16,000개
    validation_steps=200,   # batch_size(20) x validation_steps(200) = 4000, 검증 데이터가 4,000개
    callbacks=[mc, es]
).fit(
    train_generator,
    validation_data=valid_generator,
    epochs=100,
    steps_per_epoch=800,    # batch_size(20) x steps_per_epoch(800) = 16000, 트레인 데이터가 16,000개
    validation_steps=200,   # batch_size(20) x validation_steps(200) = 4000, 검증 데이터가 4,000개
    callbacks=[mc, es]
)

Epoch 1/100
 11/800 [..............................] - ETA: 4:56 - loss: 0.6931 - accuracy: 0.5045

KeyboardInterrupt: ignored

In [ ]:
best_model = load_model(model_path)
best_model.evaluate(test_generator)

In [ ]:
from google.colab import files

up = files.upload()

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('cat02.jpg')[:,:,::-1]
img2 = cv2.resize(img, dsize=(244,244), interpolation=cv2.INTER_AREA)
plt.figure(figsize=(10, 3))
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(img2)
plt.show()